In [1]:
## Import Packages
from __future__ import print_function

import numpy as np
import pandas as pd
from itertools import product

#Astro Software
import astropy.units as units
from astropy.coordinates import SkyCoord
from astropy.io import fits

#Plotting Packages
import matplotlib as mpl
import matplotlib.cm as cmplt
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

import seaborn as sns

from PIL import Image, ImageDraw

from yt.config import ytcfg
import yt
import yt.units as u

#Scattering NN
import torch
import torch.nn.functional as F
from torch import optim
from kymatio.torch import Scattering2D
device = "cpu"

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import skimage
from skimage import filters
from skimage.filters import window

from scipy.optimize import curve_fit
from scipy import linalg
from scipy import stats
from scipy.signal import general_gaussian
from scipy.ndimage import map_coordinates

# from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee

#I/O
import h5py
import pickle
import glob
import copy
import time
import os
#from bs4 import BeautifulSoup
import scipy.io as sio

#Plotting Style
%matplotlib inline
plt.style.use('dark_background')
rcParams['text.usetex'] = False
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['legend.fontsize'] = 12
rcParams['axes.labelsize'] = 20
rcParams['font.family'] = 'sans-serif'

#Threading
torch.set_num_threads=2
from multiprocessing import Pool

import ntpath
def path_leaf(path):
    head, tail = ntpath.split(path)
    out = os.path.splitext(tail)[0]
    return out

def hd5_open(file_name,name):
    f=h5py.File(file_name,'r', swmr=True)
    data = f[name][:]
    f.close()
    return data

from matplotlib.colors import LinearSegmentedColormap
cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
blue_red1 = LinearSegmentedColormap('BlueRed1', cdict1,N=5000)
blue_red1.set_bad(color='yellow')

In [2]:
def DHC_iso_all(wst,J,L):
    (nk, Nd) = np.shape(wst)
    S0 = wst[:,0:2]
    S1 = wst[:,2:J*L+2]
    S2 = np.transpose(np.reshape(wst[:,J*L+3:],(nk,(J*L+1),(J*L+1))),(0,2,1))
    
    S1iso = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            S1iso[:,j1] += S1[:,l1*J+j1]
    
    S2iso = np.zeros((nk,J,J,L))
    for j1 in range(J):
        for j2 in range(J):
            for l1 in range(L):
                for l2 in range(L):
                    deltaL = np.mod(l1-l2,L)
                    S2iso[:,j1,j2,deltaL] += S2[:,l1*J+j1,l2*J+j2]
                    
    Sphi1 = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            Sphi1[:,j1] += S2[:,l1*J+j1,L*J]
            
    Sphi2 = np.zeros((nk,J))
    for j1 in range(J):
        for l1 in range(L):
            Sphi2[:,j1] += S2[:,L*J,l1*J+j1]
            
    return np.hstack((S0,S1iso,wst[:,J*L+2].reshape(nk,1),S2iso.reshape(nk,int(J*J*L)),Sphi1,Sphi2,S2[:,L*J,L*J].reshape(nk,1)))

In [3]:
mnist_DHC_out = hd5_open('mnist_DHC_out.h5','main/data')
mnist_DHC_out_test = hd5_open('mnist_DHC_out_test.h5','main/data')

In [4]:
mnist_train_y = hd5_open('mnist_train_y.h5','main/data')
mnist_test_y = hd5_open('mnist_test_y.h5','main/data')

In [5]:
def prec_LDA(lda,X_test,y_test,X_train,y_train):
    y_pred = lda.predict(X_test)
    prec = precision_score(y_test,y_pred,average='micro')
    
    y_train_pred = lda.predict(X_train)
    prec_train = precision_score(y_train,y_train_pred,average='micro')
    return (prec, prec_train)

In [6]:
temp = DHC_iso_all(mnist_DHC_out,6,8)
temp_test = DHC_iso_all(mnist_DHC_out_test,6,8)

combo_X = np.vstack((temp,temp_test))
combo_Y = np.hstack((mnist_train_y,mnist_test_y))

In [7]:
def mask_prec_fn(j,mask,X_train,y_train,X_test,y_test):
    mask_loc = copy.deepcopy(mask)
    mask_loc[j] = False
    lda = LDA()
    lda.fit(X_train[:,mask_loc], y_train)
    prec, prec_train = prec_LDA(lda,X_test[:,mask_loc],y_test,X_train[:,mask_loc],y_train)
    return (j,prec, prec_train)

In [8]:
from functools import partial

In [9]:
import tqdm as tq

In [ ]:
k=0
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(train_prec_temp == train_prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_test_{}.h5'.format(k), 'w')
        hf.create_dataset('drop_indx', data=drop_indx)
        hf.create_dataset('prec_max', data=prec_max)
        hf.create_dataset('train_prec', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

0


  0%|          | 1/309 [00:56<4:50:49, 56.65s/it]

296 0.9284285714285714 0.9290178571428571


  1%|          | 2/309 [01:53<4:50:39, 56.80s/it]

...

225 0.9285714285714286 0.9295535714285714


  6%|▌         | 17/309 [15:55<4:28:25, 55.15s/it]

57 0.9285714285714286 0.9295535714285714


  6%|▌         | 18/309 [16:49<4:26:23, 54.92s/it]

63 0.9279285714285714 0.9295892857142857


In [ ]:
k=2
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(train_prec_temp == train_prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_test_{}.h5'.format(k), 'w')
        hf.create_dataset('drop_indx', data=drop_indx)
        hf.create_dataset('prec_max', data=prec_max)
        hf.create_dataset('train_prec', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

24306


  0%|          | 1/309 [01:13<6:19:11, 73.87s/it]

271 0.928 0.9289642857142857


  1%|          | 2/309 [02:34<6:28:24, 75.91s/it]

...

121 0.9002857142857142 0.8989821428571428


 84%|████████▍ | 261/309 [3:03:08<16:32, 20.67s/it]

284 0.8995 0.8979642857142857


 85%|████████▍ | 262/309 [3:03:28<15:58, 20.39s/it]

6 0.898 0.8967857142857143


In [ ]:
k=3
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(train_prec_temp == train_prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_test_{}.h5'.format(k), 'w')
        hf.create_dataset('drop_indx', data=drop_indx)
        hf.create_dataset('prec_max', data=prec_max)
        hf.create_dataset('train_prec', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

36459


  0%|          | 1/309 [01:01<5:15:41, 61.50s/it]

273 0.93 0.9287857142857143


  1%|          | 2/309 [02:01<5:12:54, 61.15s/it]

...

231 0.9097857142857143 0.9097857142857143


 79%|███████▉  | 244/309 [3:03:20<32:52, 30.35s/it]

119 0.9093571428571429 0.9091964285714286


 79%|███████▉  | 245/309 [3:03:50<32:16, 30.26s/it]

284 0.9093571428571429 0.9085714285714286


In [ ]:
k=5
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(train_prec_temp == train_prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_test_{}.h5'.format(k), 'w')
        hf.create_dataset('drop_indx', data=drop_indx)
        hf.create_dataset('prec_max', data=prec_max)
        hf.create_dataset('train_prec', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

60765


  0%|          | 1/309 [01:21<6:57:20, 81.30s/it]

275 0.9299285714285714 0.9288035714285714


  1%|          | 2/309 [02:42<6:56:09, 81.33s/it]

...

145 0.9267857142857143 0.9293928571428571


 11%|█▏        | 35/309 [44:35<5:00:57, 65.90s/it]

269 0.9267857142857143 0.9293928571428571


 12%|█▏        | 36/309 [45:44<5:04:57, 67.02s/it]

143 0.9267857142857143 0.9293928571428571


In [ ]:
k=9
for n in range(10):
    if __name__ == '__main__':
        pool = Pool(31)
        print(int(k*12153))
        X_train, X_test, y_train, y_test = train_test_split(
            combo_X, 
            combo_Y, 
            stratify=combo_Y, 
            train_size=0.8,
            test_size=0.2,
            random_state=int(k*12153))

        index_list = np.arange(310)

        prec_max = []
        drop_indx = []
        train_prec = []

        for i in tq.tqdm(range(309),leave=True):
            mask = np.ones(310, dtype=bool)
            mask[drop_indx] = False
            prec_temp = np.zeros(310)
            train_prec_temp = np.zeros(310)
            temp_fnc = partial(mask_prec_fn, mask=mask,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)
            out = pool.map(temp_fnc,index_list[mask])
            out1 = list(map(list, zip(*out)))

            for indx, ind in enumerate(out1[0]):
                prec_temp[ind] = out1[1][indx]
                train_prec_temp[ind] = out1[2][indx]

            arg = np.random.choice(np.flatnonzero(train_prec_temp == train_prec_temp.max()))
            drop_indx.append(arg)
            prec_arg = prec_temp[arg]
            prec_max.append(prec_arg)
            train_prec_arg = train_prec_temp[arg]
            train_prec.append(train_prec_arg)

            print(arg,prec_arg,train_prec_arg)

        pool.close()
        pool.join()

        hf = h5py.File('drop_test_{}.h5'.format(k), 'w')
        hf.create_dataset('drop_indx', data=drop_indx)
        hf.create_dataset('prec_max', data=prec_max)
        hf.create_dataset('train_prec', data=train_prec)
        hf.close()

        k+=1

  0%|          | 0/309 [00:00<?, ?it/s]

109377


  0%|          | 1/309 [01:00<5:11:36, 60.70s/it]

258 0.9266428571428571 0.929


  1%|          | 2/309 [02:01<5:11:11, 60.82s/it]

...

305 0.9247142857142857 0.9308392857142858


 15%|█▌        | 47/309 [42:09<3:28:22, 47.72s/it]

306 0.9247142857142857 0.9308392857142858


 16%|█▌        | 48/309 [42:56<3:26:33, 47.49s/it]

269 0.9247142857142857 0.9308392857142858
